In [ ]:
import sys
from pathlib import Path

ROOT_DIR = Path.cwd().parent
sys.path.append(str(ROOT_DIR))

print("DEBUG: Root added to path:", ROOT_DIR)

In [ ]:
import os
import json
# import sqlite3
from dotenv import load_dotenv
from pathlib import Path
from openai import OpenAI
import pandas as pd
import gradio as gr

load_dotenv(override=True)

if os.getenv("NVIDIA_API_KEY"):
    print("DEBUG: OPENAI_API_KEY Loaded")
else:
    print("DEBUG: OPENAI_API_KEY NOT FOUND - please set in .env")

# Initialize OpenAI client and model
client = OpenAI()
MODEL = "qwen/qwen3-next-80b-a3b-instruct"  


In [ ]:
SYSTEM_PROMPT = """
You are a chatbot for PVR Cinemas. You can ONLY answer questions using the data provided to you. 
Do NOT make up information.

Rules:
- If the product does not exist, say: "This product is not available in PVR inventory."
- If the product exists but warehouse data is not available, say so clearly.
- If the product quantity is 0, mention that it is out of stock and show warehouse options with prices.
- If the product is in stock, show its quantity.
- Keep responses short and clear.
"""

In [ ]:
inventory_data = [
    {"product_id": "PVR001", "product_name": "Salted Popcorn (Large)", "category": "Snacks", "quantity": 120, "retail_price": 450, "reorder_level": 30},
    {"product_id": "PVR002", "product_name": "Cheese Popcorn (Large)", "category": "Snacks", "quantity": 0, "retail_price": 550, "reorder_level": 25},
    {"product_id": "PVR003", "product_name": "Caramel Popcorn (Large)", "category": "Snacks", "quantity": 35, "retail_price": 550, "reorder_level": 25},
    {"product_id": "PVR004", "product_name": "Nachos with Cheese Dip", "category": "Snacks", "quantity": 0, "retail_price": 620, "reorder_level": 20},
    {"product_id": "PVR005", "product_name": "Veg Burger", "category": "Food", "quantity": 20, "retail_price": 280, "reorder_level": 15},
    {"product_id": "PVR006", "product_name": "Chicken Burger", "category": "Food", "quantity": 15, "retail_price": 350, "reorder_level": 10},
    {"product_id": "PVR007", "product_name": "French Fries (Large)", "category": "Snacks", "quantity": 50, "retail_price": 220, "reorder_level": 20},
    {"product_id": "PVR008", "product_name": "Cold Drink (Coke 500ml)", "category": "Beverages", "quantity": 80, "retail_price": 180, "reorder_level": 40},
    {"product_id": "PVR009", "product_name": "Cold Drink (Sprite 500ml)", "category": "Beverages", "quantity": 60, "retail_price": 180, "reorder_level": 40},
    {"product_id": "PVR010", "product_name": "Chocolate Brownie", "category": "Desserts", "quantity": 10, "retail_price": 250, "reorder_level": 5},
]

inventory_df = pd.DataFrame(inventory_data)
print("Inventory DataFrame created:")
# print(inventory_df)

In [ ]:
warehouse_data = [
    {"warehouse_id": "WH001", "warehouse_name": "Mumbai Central Warehouse", "product_id": "PVR002", "price": 165, "latitude": 19.0760, "longitude": 72.8777},
    {"warehouse_id": "WH002", "warehouse_name": "Andheri East Warehouse", "product_id": "PVR002", "price": 170, "latitude": 19.1136, "longitude": 72.8697},
    {"warehouse_id": "WH003", "warehouse_name": "Bandra Warehouse", "product_id": "PVR004", "price": 210, "latitude": 19.0596, "longitude": 72.8295},
    {"warehouse_id": "WH004", "warehouse_name": "Lower Parel Warehouse", "product_id": "PVR004", "price": 205, "latitude": 19.0033, "longitude": 72.8292},
    {"warehouse_id": "WH005", "warehouse_name": "Kurla Warehouse", "product_id": "PVR001", "price": 130, "latitude": 19.0726, "longitude": 72.8825},
    {"warehouse_id": "WH006", "warehouse_name": "Powai Warehouse", "product_id": "PVR001", "price": 135, "latitude": 19.1176, "longitude": 72.9060},
    {"warehouse_id": "WH007", "warehouse_name": "Vashi Warehouse", "product_id": "PVR003", "price": 150, "latitude": 19.0750, "longitude": 72.9980},
    {"warehouse_id": "WH008", "warehouse_name": "Chembur Warehouse", "product_id": "PVR003", "price": 148, "latitude": 19.0522, "longitude": 72.9005},
    {"warehouse_id": "WH009", "warehouse_name": "Goregaon Warehouse", "product_id": "PVR005", "price": 95, "latitude": 19.1663, "longitude": 72.8526},
    {"warehouse_id": "WH010", "warehouse_name": "Malad Warehouse", "product_id": "PVR005", "price": 92, "latitude": 19.1875, "longitude": 72.8484},
    {"warehouse_id": "WH011", "warehouse_name": "Kandivali Warehouse", "product_id": "PVR006", "price": 135, "latitude": 19.2094, "longitude": 72.8526},
    {"warehouse_id": "WH012", "warehouse_name": "Borivali Warehouse", "product_id": "PVR006", "price": 138, "latitude": 19.2307, "longitude": 72.8567},
    {"warehouse_id": "WH013", "warehouse_name": "Dadar Warehouse", "product_id": "PVR007", "price": 110, "latitude": 19.0176, "longitude": 72.8562},
    {"warehouse_id": "WH014", "warehouse_name": "Sion Warehouse", "product_id": "PVR007", "price": 108, "latitude": 19.0434, "longitude": 72.8630},
    {"warehouse_id": "WH015", "warehouse_name": "Thane Warehouse", "product_id": "PVR008", "price": 48, "latitude": 19.2183, "longitude": 72.9781},
    {"warehouse_id": "WH016", "warehouse_name": "Mulund Warehouse", "product_id": "PVR008", "price": 50, "latitude": 19.1726, "longitude": 72.9565},
    {"warehouse_id": "WH017", "warehouse_name": "Bhandup Warehouse", "product_id": "PVR009", "price": 47, "latitude": 19.1511, "longitude": 72.9372},
    {"warehouse_id": "WH018", "warehouse_name": "Ghatkopar Warehouse", "product_id": "PVR009", "price": 49, "latitude": 19.0856, "longitude": 72.9081},
    {"warehouse_id": "WH019", "warehouse_name": "Colaba Warehouse", "product_id": "PVR010", "price": 75, "latitude": 18.9067, "longitude": 72.8147},
    {"warehouse_id": "WH020", "warehouse_name": "Fort Warehouse", "product_id": "PVR010", "price": 78, "latitude": 18.9322, "longitude": 72.8347},
]

warehouse_df = pd.DataFrame(warehouse_data)
print("Warehouse DataFrame created:")
# print(warehouse_df)

In [ ]:
def df_to_context(df, title):
    """Convert DataFrame to readable context string"""
    rows = df.to_dict(orient="records")
    context = f"{title}:\n"
    for r in rows:
        context += f"{r}\n"
    return context


In [ ]:
def chat_llm(message):
    """Send message to LLM and get response"""
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": message}
        ],
        temperature=0.7
    )
    return response.choices[0].message.content

def pvr_chatbot(user_query):
    """Main chatbot function that combines data and queries LLM"""
    inventory_context = df_to_context(inventory_df, "Inventory Data")
    warehouse_context = df_to_context(warehouse_df, "Warehouse Data")
    
    message = f"""
{inventory_context}

{warehouse_context}

User Question: {user_query}
"""
    return chat_llm(message)

In [ ]:
pvr_chatbot("cheese popcorn is out of stock, what are its price details in the warehouse")

In [ ]:
def gradio_chat(user_message, history):
    """
    Simple chat function for gr.ChatInterface
    - user_message: str (current user input)
    - history: list of previous messages (handled automatically by ChatInterface)
    Returns: bot response as string
    """
    bot_response = pvr_chatbot(user_message)
    return bot_response

# Cell 9: Launch Gradio App
view = gr.ChatInterface(
    fn=gradio_chat,
    title="🎬 PVR Cinemas Inventory Chatbot",
    description="Ask about product availability, stock, or vendor prices. Responses are based only on internal PVR data.",
    examples=[
        "Is Cheese Popcorn available?",
        "What's the price of Veg Burger?",
        "Show me all beverages in stock",
        "Where can I get Nachos with Cheese Dip?"
    ],
)

print("DEBUG: Gradio view prepared. Call view.launch() to start UI.")
view.launch(debug=True)

In [ ]:
pvr_chatbot("cheese popcorn is out of stock, what are its price details in the warehouse")